# 쇼핑몰 매출 예측 (Times Series) - Part1

시계열 분석 진행 방향
- 시간 관련 데이터 다양하게 조작하기
- fbprophet을 통한 시계열 예측
- AutoRegressive 알고리즘을 통한 예측

# 1. 시간 관련 데이터 다양하게 조작하기

In [2]:
import pandas as pd 
import numpy as np

# 시간과 관련된 모듈
from datetime import datetime, timedelta

In [4]:
# 현재 날짜
today = datetime.now()
today

datetime.datetime(2024, 10, 19, 13, 4, 44, 650630)

In [9]:
# 날짜 쪼개기
print(today.year)
print(today.month)
print(today.day)
print('------------------')

print(today.hour)
print(today.minute)
print(today.second)
print(today.microsecond)

2024
10
19
------------------
13
4
44
650630


In [10]:
# 날짜 계산
today + timedelta(days=10)

datetime.datetime(2024, 10, 29, 13, 4, 44, 650630)

In [11]:
today + timedelta(weeks=3)

datetime.datetime(2024, 11, 9, 13, 4, 44, 650630)

In [13]:
df = pd.DataFrame({'date' : ['10/19/2024', '10/20/2024', '10/21/2024']
                   , 'value' : [10, 20, 30]})

df

,date,value
0,10/19/2024,10
1,10/20/2024,20
2,10/21/2024,30


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   value   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [16]:
## 날짜형 데이터 타입 변환
df['date'] = pd.to_datetime(df['date'])

# 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
 1   value   3 non-null      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 180.0 bytes


In [18]:
df['date'].dt.month

0    10
1    10
2    10
Name: date, dtype: int32

In [20]:
df = pd.DataFrame({'date' : ['11-01-2024 1:11:11', '11-02-2024 2:11:11', '11-03-2024 3:11:11']
                   , 'value' : [10, 20, 30]})

df

,date,value
0,11-01-2024 1:11:11,10
1,11-02-2024 2:11:11,20
2,11-03-2024 3:11:11,30


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   value   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [22]:
pd.to_datetime(df['date'])

0   2024-11-01 01:11:11
1   2024-11-02 02:11:11
2   2024-11-03 03:11:11
Name: date, dtype: datetime64[ns]

In [23]:
## 날짜형 데이터 타입 변환
df['date'] = pd.to_datetime(df['date'])

# 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
 1   value   3 non-null      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 180.0 bytes


In [24]:
df['date'].dt.hour

0    1
1    2
2    3
Name: date, dtype: int32

In [25]:
df = pd.DataFrame({'date' : ['24-11-10', '24-11-11', '24-11-12']
                   , 'value' : [10, 20, 30]})

df

,date,value
0,24-11-10,10
1,24-11-11,20
2,24-11-12,30


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   value   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [27]:
# 날짜형 데이터 타입 변환 (month/day/year로 저장)
pd.to_datetime(df['date'])

C:\Users\yuzln\AppData\Local\Temp\ipykernel_162448\3273306264.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df['date'])


0   2010-11-24
1   2011-11-24
2   2012-11-24
Name: date, dtype: datetime64[ns]

In [30]:
# 날짜형 포멧 지정
pd.to_datetime(df['date'], format='%y-%m-%d')

0   2024-11-10
1   2024-11-11
2   2024-11-12
Name: date, dtype: datetime64[ns]

In [40]:
# 데이터 불러오기
df = pd.read_excel("Superstore.xls")

print(df.shape)
df.head(3)

(9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [41]:
# Indexing
df = df[['Order Date', 'Ship Date', 'Sales', 'Profit']].copy()

print(df.shape)
df.head()

(9994, 4)


,Order Date,Ship Date,Sales,Profit
0,2016-11-08,2016-11-11,261.9600,41.9136
1,2016-11-08,2016-11-11,731.9400,219.5820
2,2016-06-12,2016-06-16,14.6200,6.8714
3,2015-10-11,2015-10-18,957.5775,-383.0310
4,2015-10-11,2015-10-18,22.3680,2.5164


In [42]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order Date  9994 non-null   datetime64[ns]
 1   Ship Date   9994 non-null   datetime64[ns]
 2   Sales       9994 non-null   float64       
 3   Profit      9994 non-null   float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 312.4 KB


In [43]:
## 날짜 차이
df['gap'] = df['Ship Date'] - df['Order Date']

# 확인
df.head()

,Order Date,Ship Date,Sales,Profit,gap
0,2016-11-08,2016-11-11,261.9600,41.9136,3 days
1,2016-11-08,2016-11-11,731.9400,219.5820,3 days
2,2016-06-12,2016-06-16,14.6200,6.8714,4 days
3,2015-10-11,2015-10-18,957.5775,-383.0310,7 days
4,2015-10-11,2015-10-18,22.3680,2.5164,7 days


In [46]:
# 데이터 타입 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   Order Date  9994 non-null   datetime64[ns] 
 1   Ship Date   9994 non-null   datetime64[ns] 
 2   Sales       9994 non-null   float64        
 3   Profit      9994 non-null   float64        
 4   gap         9994 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), timedelta64[ns](1)
memory usage: 390.5 KB


# Linear Regression

In [47]:
from sklearn.linear_model import LinearRegression

# timedelta가 linear regression에서 독립변수로 역할을 충분히 할 수 있을까?
X = df[['Profit', 'gap']]
y = df['Sales']

In [48]:
# 객체 생성
model = LinearRegression()

# 모델 적합
# 에러가 남 → timedelta를 독립변수로 받지 못함
model.fit(X, y)

TypeError: The DType <class 'numpy.dtype[float64]'> could not be promoted by <class 'numpy.dtype[timedelta64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[float64]'>, <class 'numpy.dtype[timedelta64]'>)

In [53]:
## 방법 1) gap 변수에 있는 숫자만 불러와 integer로 변환
X['gap'] = X['gap'].dt.days

# 확인
X.head()

C:\Users\yuzln\AppData\Local\Temp\ipykernel_162448\1832922316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['gap'] = X['gap'].dt.days


,Profit,gap
0,41.9136,3
1,219.5820,3
2,6.8714,4
3,-383.0310,7
4,2.5164,7


In [55]:
# 객체 생성
model = LinearRegression()

# 모델 적합
model.fit(X, y)

# 모델 예측
model.predict(X)

array([248.5051552 , 474.93985046, 202.01637137, ..., 216.14692021,
       208.40675263, 284.40140338])

In [57]:
## 방법 2) unix time stamp로 데이터 형식 변환
df.head()

,Order Date,Ship Date,Sales,Profit,gap
0,2016-11-08,2016-11-11,261.9600,41.9136,3 days
1,2016-11-08,2016-11-11,731.9400,219.5820,3 days
2,2016-06-12,2016-06-16,14.6200,6.8714,4 days
3,2015-10-11,2015-10-18,957.5775,-383.0310,7 days
4,2015-10-11,2015-10-18,22.3680,2.5164,7 days


In [63]:
## unix time stamp : 모든 시간 단위를 'second'로 변환
pd.set_option('display.float_format', '{:.2f}'.format)

# microsecond를 제외하기 위해 10의 9승으로 나눔
df['Order Date'] = pd.to_numeric(df['Order Date']) / 10**9
df['Ship Date'] = pd.to_numeric(df['Ship Date']) / 10**9

# 확인
df.head()

,Order Date,Ship Date,Sales,Profit,gap
0,1478563200.00,1478822400.00,261.96,41.91,3 days
1,1478563200.00,1478822400.00,731.94,219.58,3 days
2,1465689600.00,1466035200.00,14.62,6.87,4 days
3,1444521600.00,1445126400.00,957.58,-383.03,7 days
4,1444521600.00,1445126400.00,22.37,2.52,7 days


In [65]:
## 날짜 차이
df['gap_1'] = df['Ship Date'] - df['Order Date']

# 확인
df.head()

,Order Date,Ship Date,Sales,Profit,gap,gap_1
0,1478563200.00,1478822400.00,261.96,41.91,3 days,259200.00
1,1478563200.00,1478822400.00,731.94,219.58,3 days,259200.00
2,1465689600.00,1466035200.00,14.62,6.87,4 days,345600.00
3,1444521600.00,1445126400.00,957.58,-383.03,7 days,604800.00
4,1444521600.00,1445126400.00,22.37,2.52,7 days,604800.00


In [66]:
from sklearn.linear_model import LinearRegression

X = df[['Profit', 'gap_1']]
y = df['Sales']

# 객체 생성
model = LinearRegression()

# 모델 적합
model.fit(X, y)

# 모델 예측
model.predict(X)

array([248.5051552 , 474.93985046, 202.01637137, ..., 216.14692021,
       208.40675263, 284.40140338])